In [ ]:
# %load defaults.ipy
from numpy import *
import numpy as np
import matplotlib
matplotlib.rcParams['savefig.dpi'] = 600
%matplotlib inline

import matplotlib.pyplot as plt

import sys
import os
import os.path
sys.path.append('../python')
import plot_info
from plot_info import showAndSave
import netCDF4
from IPython.core.display import display, HTML
import tikzplotlib
import copy
import json
import traceback
with open('structure_function_configs.json') as f:
    configurations = json.load(f)

def for_each_config(f):
    oldPrefix = copy.deepcopy(showAndSave.prefix)
    for conf in configurations.keys():
        
        plot_info.console_log(conf)
        headline(configurations[conf]['name'])
        showAndSave.prefix = oldPrefix + conf
        try:
            f(configurations[conf])
        except Exception as e:
            plot_info.console_log("Skipping {fname} on {conf} ({confname})\n\tReason: {reason}\nTraceback: {traceback}".format(fname=str(f),
                                                                                                       conf=conf, 
                                                                                                       confname=configurations[conf]['name'],
                                                                                                       reason=str(e),
                                                                                                     traceback=traceback.format_exc()))
    showAndSave.prefix=oldPrefix

plot_info.set_notebook_name("StructureAlsvinn.ipynb")

print("STATISTICAL_STRUCTURE={}".format(plot_info.get_environment("STATISTICAL_STRUCTURE", [])))

In [ ]:
def headline(text):
    display(HTML("<h1>%s</h1>"%text))
def h2(text):
    display(HTML("<h2>%s</h2>"%text))
def h3(text):
    display(HTML("<h3>%s</h3>"%text))
def h4(text):
    display(HTML("<h4>%s</h4>"%text))
def h5(text):
    display(HTML("<h4>%s</h4>"%text))

In [ ]:
def readnetcdf4(filename, variable):
    basepath = plot_info.get_environment("STATISTICAL_STRUCTURE", [filename])
    with netCDF4.Dataset(os.path.join(basepath, filename)) as f:
        return f.variables[variable][:,0,0]

def upscale(d, r):

    while d.shape[0] < r:
       
        d = repeat(d,2,0)
    return d

def plotscalings_perturbation(d,pert, title,plot_nonlog=True,name=''):
    N = len(d)
    H = linspace(0, 64./2048,N)
   
    poly = polyfit(log(H[1:]),log(d[1:]),1)
  
  
        
    fig = plt.figure(0,figsize=2*plt.figaspect(0.9))
    plt.title(title)
    ax = plt.gca()
    ax.loglog(H[1:],d[1:],label='%s $\\epsilon=%.4f$'%(name, pert))
    ax.loglog(H[1:], exp(poly[1])*H[1:]**(poly[0]), '--',label='$\\mathcal{O}(h^{%.2f})$' % poly[0])
    box = ax.get_position()
    #ax.set_position([box.x0, box.y0 + box.height * 0.1,
    #                 box.width, box.height * 0.9])
    #
    ## Put a legend below current axis
    #ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
    #      fancybox=True, shadow=True, ncol=5)
    ax.legend(loc='center right', bbox_to_anchor=(1, 0.5))
    ax.grid('on')



def plotscalings(d,r,title,plot_nonlog=True):
    N = len(d)
    H = linspace(0, 64./2048,N)
   
    poly = polyfit(log(H[1:]),log(d[1:]),1)
    if plot_nonlog:
        fig = plt.figure(0, figsize=2*plt.figaspect(0.5))
        fig.suptitle(title)
        ax = fig.add_subplot(1,2,1)
        
        ax.plot(H,d,label='r=%d'%r)
        ax.plot(H, exp(poly[1])*H**(poly[0]), '--',label='$\\mathcal{O}(h^{%.2f})$' % poly[0])
        ax.grid('on')
        box = ax.get_position()
        ax.set_position([box.x0, box.y0 + box.height * 0.1,
                         box.width, box.height * 0.9])
        
        # Put a legend below current axis
        #ax.legend(pos='upper center', bbox_to_anchor=(0.5, -0.05),
        #     fancybox=True, shadow=True, ncol=5)
        
        ax.legend(bbox_to_anchor=(0.5, -0.05),
             fancybox=True, shadow=True, ncol=5)
        
    
          

        ax = fig.add_subplot(1,2,2)
    else:
        
        fig = plt.figure(0,figsize=2*plt.figaspect(0.9))
        plt.title(title)
        ax = plt.gca()
    ax.loglog(H[1:],d[1:],label='$N=%d$'%r)
    ax.loglog(H[1:], exp(poly[1])*H[1:]**(poly[0]), '--',label='$\\mathcal{O}(h^{%.2f})$' % poly[0])
    box = ax.get_position()
    #ax.set_position([box.x0, box.y0 + box.height * 0.1,
    #                 box.width, box.height * 0.9])
    #
    ## Put a legend below current axis
    #ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
    #      fancybox=True, shadow=True, ncol=5)
    ax.legend(loc='center right', bbox_to_anchor=(1, 0.5))
    ax.grid('on')




def getlineintegral(filename, variable,resolution):
    d = readnetcdf4(filename, variable)
    dx = 1.0/resolution
    
    for h in range(1,len(d)):
        d[h] += d[h-1]
    for h in range(1,len(d)):
        d[h] /=(2*h+1)**2
        #d[h] /= 4*(2*h+1)
        
    return d
    

# Mean and variance

In [ ]:
def plot_mean_var_convergence(configuration):
    perturbations = configuration['perturbations']
    Tarray = configuration['Tarray']
    resolutions = configuration['resolutions']
    stat_filename = configuration['stat_filename']
    name = configuration['name']
    maxT = configuration['maxT']
    
    # Here we only care about certain configurations (others not relevant)
    if 'richtmeyer' not in name.lower() and 'kelvin' not in name.lower() and 'brownian' not in name.lower():
        return
    for perturbation in perturbations:
        timeArray = [Tarray[0], Tarray[-1]]
        if 'kelvin' in name.lower():
            timeArray = Tarray
            
        if Tarray[1] not in timeArray:
            timeArray.append(Tarray[1])
        for time in timeArray:
            for stat in ['mean', 'variance']:
                errors = zeros(len(resolutions)-1)
                for var in ['rho','E','mx','my']:
                    for (nr, r) in enumerate(resolutions):
                        filename = stat_filename.format(stat=stat,t=time, perturbation=perturbation, resolution=r)
                        
                        basepath = plot_info.get_environment("STATISTICAL_STRUCTURE", [filename])
                        #print stat_filename.format(stat=stat,t=time, perturbation=perturbation, resolution=r)
                        with netCDF4.Dataset(os.path.join(basepath, filename)) as f:
                            d = f.variables[var][:,:,0].T
                            N = d.shape[0]


                            if r > resolutions[0]:
                                errors[nr-1] += (sum(abs(d-dPrev))/r**2)
                            dPrev = repeat(d,2,0)
                            dPrev = repeat(dPrev,2,1)
                            if var == 'rho':
                                if stat == 'variance':
                                    statName = '\\mathrm{Var}'
                                elif stat == 'mean':
                                    statName = '\\mathbb{E}'
                                x,y = mgrid[0:1:N*1j,0:1:N*1j]
                                plt.pcolormesh(x,y,d)
                                plt.colorbar()
                                if len(perturbations) > 1:
                                    plt.title('{name}, ${stat} (\\rho)$, $t={time}$, $\\varepsilon={perturbation}$'.format(name=name, time=maxT*time/(float(max(Tarray))), stat=statName, perturbation=perturbation))
                                else:
                                    plt.title('{name}, ${stat}(\\rho)$, $t={time}$'.format(name=name,time=maxT*time/(float(max(Tarray))), stat=statName, perturbation=perturbation))


                                plt.xlabel('$x$')
                                plt.ylabel('$y$')
                                showAndSave('%s_%s_%d_%.4f_%d' %(stat,name.replace('$','').replace(' ','_').replace('=',''), r,perturbation,time))
                print (errors )
                plt.xlim(resolutions[0], resolutions[-2])  
                plt.ylim(min(errors),max(errors))
                plt.loglog(resolutions[:-1], errors, '-o', basex=2, basey=2)
                plt.xlabel('Resolution')
                if stat == 'mean':
                    plt.ylabel('$||\\mathbb{E}(u^{2\Delta x})-\\mathbb{E}(u^{\Delta x})||_{L^1(D,\\mathbb{R}^4)}$')
                elif stat == 'variance':
                    plt.ylabel('$||\\mathrm{Var}(u^{2\Delta x})-\\mathrm{Var}(u^{\Delta x})||_{L^1(D,\\mathbb{R}^4)}$')

                if len(perturbations)>1:
                    plt.title('Convergence {name}, {stat}, $t={time}$, $\\varepsilon={perturbation}$'.format(name=name,time=maxT*time/(float(max(Tarray))), stat=stat, perturbation=perturbation))
                else:
                    plt.title('Convergence {name}, {stat}, $t={time}$'.format(name=name, time=maxT*time/(float(max(Tarray))), stat=stat, perturbation=perturbation))
                plt.xticks(resolutions[:-1], ['${r} \\times {r}$'.format(r=r) for r in resolutions[:-1]])

                plt.grid(True)
                showAndSave('convergence_%s_%s_%.4f_%d' %(stat,name.replace('$','').replace(' ','_').replace('=','') ,perturbation,time))
                
for_each_config(plot_mean_var_convergence)

In [ ]:
def convergence_structure_functions(config):
    headline("Component convergence Box averaged structure functions")
    variables = ['rho', 'mx','my','E']
    perturbations = config['perturbations']
    Tarray = config['Tarray']
    resolutions = config['resolutions']
    cube_filename = config['cube_filename']
    name = config['name']
    maxT = config['maxT']
    for perturbation in perturbations:
        h2("perturbation = %.2f" % perturbation)
        for p in [1,2,3]:
            h3("p = %d" % p)
            timeArray = [Tarray[-1]]
            if 'kelvin' in name.lower():
                timeArray = Tarray
            try:
                if Tarray[1] not in timeArray:
                    timeArray.append(Tarray[1])
            except:
                pass
                
            if 1 not in Tarray:
                timeArray.append(1)
            for t in timeArray:
                h4("t = %d" % (0.2*t))

                errors = zeros(len(resolutions)-1)


                for (nr, resolution) in enumerate(resolutions):
                    dVar = zeros((64*resolution)//2048)
                    for variable in variables:

                        filename = cube_filename.format(perturbation = perturbation, p=p,t=t,resolution=resolution)

                        d = getlineintegral(filename, variable, resolution)


                        dVar += d
                    dVar = dVar**(1.0/p)
                    if resolution > resolutions[0]:
                        errors[nr-1] = abs(dVar[-1]-dPrev[-1])
                    dPrev = repeat(dVar,2,0)
                    if len(perturbations) > 1:
                        title = 'Scaling for {name}, $t={time},p={p},\\varepsilon={perturbation}$'.format(name=name,time=t*maxT/float(len(Tarray)-1),p=p,perturbation=perturbation)
                    else:
                        title = 'Scaling for {name}, $t={time},p={p}$'.format(name=name,time=t*maxT/float(max(Tarray)),p=p,perturbation=perturbation)

                    plotscalings(dVar, resolution, title, False)
                    plt.xlabel('$h$')
                    plt.ylabel('$\\left(\\bar{S}^{%d}_{1/N,N}(h)\\right)^{\\frac{1}{%d}}$' % (p,p))
                for nr in range(len(resolutions)-1):
                    errors[nr] = abs(errors[nr])

                showAndSave('scaling_{name}_{p}_{perturbation}_{time}'.format(name=name.replace('$','').replace(' ',"_").replace('=',''), p=p, perturbation=perturbation, time=t))
                plt.loglog(resolutions[1:],errors,'-o',label='%s' % variable, basex=2, basey=2)
                
                plt.ylim([2.0**(np.floor(np.log2(min(errors))))/1.1, 1.1*2.0**(np.ceil(np.log2(max(errors))))])
                if len(perturbations)>1:
                    title = '{name}, $p={p}, t={t}, \\varepsilon={perturbation}$'.format(name=name, perturbation=perturbation, p=p,t=t*maxT/float(max(Tarray)))
                else:
                    title = '{name}, $p={p}, t={t}$'.format(name=name, perturbation=perturbation, p=p,t=t*maxT/float(max(Tarray)))

                #plt.title('{name}, $p={p}, t={t}, \\varepsilon={perturbation}$'.format(name=name, perturbation=perturbation, p=p,t=t*maxT/float(max(Tarray))))

                plt.title(title)
                plt.xlabel('Resolution')
                plt.ylabel('$|S^{%d}_{\Delta x,M}(h_{\\max})-S^{%d}_{2\Delta x,M}(h_{\\max})|$' % (p,p) )
                plt.grid('on')
                plt.xticks(resolutions[1:], ['${r} \\times {r}$'.format(r=r) for r in resolutions[1:]])
                showAndSave('convergence_{name}_{p}_{perturbation}_{time}'.format(name=name.replace('$','').replace(' ',"_").replace('=',''), p=p, perturbation=perturbation, time=t))
                
for_each_config(convergence_structure_functions)

In [ ]:
def convergence_structure_functions_perturbation(config):
    resolutions = config['resolutions']
    name = config['name']
    cube_filename = config['cube_filename']
    Tarray = config['Tarray']
    perturbations = config['perturbations']
    maxT = config['maxT']
    
    headline("Component convergence Box averaged structure functions(perturbation convergence)")
    resolution = resolutions[len(resolutions)-1]
    variables = ['rho', 'mx','my','E']

    for p in [1,2,3]:
        h3("p = %d" % p)
        for t in [Tarray[0], Tarray[1]]:
            h4("t = %d" % (0.2*t))

            errors = zeros(len(perturbations)-1)

            dVarRef = zeros((64*resolution)//2048)
            for variable in variables:

                filename = cube_filename.format(perturbation = perturbations[-1], p=p,t=t,resolution=resolution)

                d = getlineintegral(filename, variable, resolution)


                dVarRef += d
            dVarRef = dVarRef**(1.0/p)
            for (nr,perturbation) in enumerate(array(perturbations)[:-1]):
                dVar = zeros((64*resolution)//2048)
                for variable in variables:

                    filename = cube_filename.format(perturbation = perturbation, p=p,t=t,resolution=resolution)

                    d = getlineintegral(filename, variable, resolution)


                    dVar += d
                dVar = dVar**(1.0/p)
                
                errors[nr-1] = abs(dVar[-1]-dVarRef[-1])


                title = 'Scaling for {name}, $t={time},p={p}, N={N}$'.format(N=resolution, name=name,time=t*maxT/float(len(Tarray)-1),p=p)



                plt.xlabel('$h$')
                plt.ylabel('$\\left(\\bar{S}^{%d}_{1/N,N}(h)\\right)^{\\frac{1}{%d}}$' % (p,p))
              

                plotscalings_perturbation(dVar, perturbation, title, False)
            for nr in range(len(perturbations)-1):
                errors[nr] = abs(errors[nr])
            showAndSave('scaling_perturbation_{name}_{p}_{resolution}_{time}'.format(resolution=resolution, name=name.replace('$','').replace(' ',"_").replace('=',''), p=p, time=t))   

            plt.loglog(perturbations[1:],errors,'-o',label='%s' % variable, basex=2, basey=2)
            plt.title('Convergence for {name} motion (box integral, h=0.03125) , p={p}, t={t}, resolution={resolution}'.format(name=name,resolution=resolution, p=p,t=2*t))
            plt.xlabel('Perturbation ($\\epsilon$)')
            plt.ylabel('$|\\cdot|_{\\mathbb{R}^{%d}}$' % len(variables))
            plt.grid('on')
            plt.xticks(resolutions[1:], ['${r} \\times {r}$'.format(r=r) for r in resolutions[1:]])
            showAndSave('convergence_scaling_perturbation_{name}_{p}_{resolution}_{time}'.format(resolution=resolution, name=name.replace('$','').replace(' ',"_").replace('=',''), p=p, time=t))   
for_each_config(convergence_structure_functions_perturbation)

# Comparison Gaussian with uniform

In [ ]:
def compare_gaussian_uniform():
    headline("Comparison Gaussian with uniform")
    resolution = 1024
    variables = ['rho', 'mx','my','E']
    maxT=2.
    name='Kelvin-Helmholtz'
    names=['Uniform', 'Gaussian']
    Tarray=[0,1]
    resolutions = [64, 128,256,512,1024]

    perturbations = [ 0.025,0.01 ]
    variables = ['rho','mx','my','E','p','ux','uy']
    cube_filenames = ['kh_euler/kelvinhelmholtz_{perturbation}_{resolution}/kh_cube_structure_cube_{p}.000000_{t}.nc',
                      'kh_gaussian/kelvinhelmholtz_{perturbation}_{resolution}/kh_cube_structure_cube_{p}.000000_{t}.nc']

    for p in [1,2,3]:
        h3("p = %d" % p)
        for t in Tarray:
            h4("t = %d" % (0.2*t))



            for (nr,perturbation) in enumerate(array(perturbations)[:]):
                for n in range(len(cube_filenames)):
                    dVar = zeros((64*resolution)//2048)
                    for variable in variables:
                        cube_filename = cube_filenames[n]
                        filename = cube_filename.format(perturbation = perturbation, p=p,t=t,resolution=resolution)

                        d = getlineintegral(filename, variable, resolution)


                        dVar += d
                    dVar = dVar**(1.0/p)
                   


                    title = 'Scaling for {name}, $t={time},p={p}, N={N}$'.format(N=resolution, name=name,time=t*maxT/float(len(Tarray)-1),p=p)



                    plt.xlabel('$h$')
                    plt.ylabel('$\\left(\\bar{S}^{%d}_{1/N,N}(h)\\right)^{\\frac{1}{%d}}$' % (p,p))



                    plotscalings_perturbation(dVar, perturbation, title, False,name=names[n])
                
            showAndSave('scaling_compare_perturbation_{name}_{p}_{resolution}_{time}'.format(resolution=resolution, name=name.replace('$','').replace(' ',"_").replace('=',''), p=p, time=t))   
compare_gaussian_uniform()

# Comparison WENO and MC

In [ ]:
def compare_weno_mc():
    headline("Comparison WENO, MC")
    resolutions = [64, 128,256,512,1024]
    resolution = resolutions[len(resolutions)-1]
    variables = ['rho', 'mx','my','E']
    maxT=2.
    name='Kelvin-Helmholtz'
    names=['WENO2', 'MC']
    Tarray=[0,1]
    

    perturbation =0.05
    variables = ['rho','mx','my','E','p','ux','uy']
    cube_filenames = ['kh_euler/kelvinhelmholtz_0.05_{resolution}/kh_cube_structure_cube_{p}.000000_{t}.nc',
                      'kh_mc/n{resolution}/kh_cube_structure_cube_{p}.000000_{t}.nc'
                     ]

    for p in [1,2,3]:
        h3("p = %d" % p)
        for t in Tarray:
            h4("t = %d" % (0.2*t))




            for n in range(len(cube_filenames)):
                dVar = zeros((64*resolution)//2048)
                for variable in variables:
                    cube_filename = cube_filenames[n]
                    filename = cube_filename.format(perturbation = perturbation, p=p,t=t,resolution=resolution)

                    d = getlineintegral(filename, variable, resolution)


                    dVar += d
                dVar = dVar**(1.0/p)
                
                dPrev = dVar


                title = 'Scaling for {name}, $t={time},p={p}, N={N}, \\epsilon={perturbation}$'.format(perturbation=perturbation, N=resolution, name=name,time=t*maxT/float(len(Tarray)-1),p=p)



                plt.xlabel('$h$')
                plt.ylabel('$\\left(\\bar{S}^{%d}_{1/N,N}(h)\\right)^{\\frac{1}{%d}}$' % (p,p))



                plotscalings_perturbation(dVar, perturbation, title, False,name=names[n])

            showAndSave('scaling_compare_schemes_{name}_{p}_{resolution}_{time}'.format(resolution=resolution, name=name.replace('$','').replace(' ',"_").replace('=',''), p=p, time=t))   
compare_weno_mc()

# Time Evolution

In [ ]:
evolutionDatasets = {'Brownian' : {'filename' : 'evolution/brownian/brownianeuler_cube_structure_cube_{p}.000000_{t}.nc',
                                  'nSaves' : 16,
                                  'maxT' : 4},
                    'Kelvin-Helmholtz' : {'filename' : 'evolution/kh/kh_cube_structure_cube_{p}.000000_{t}.nc',
                                  'nSaves' : 8,
                                  'maxT' : 4},
                     'Richtmeyer-Meshkov' : {'filename' : 'evolution/rm/rm_cube_structure_cube_{p}.000000_{t}.nc',
                                  'nSaves' : 8,
                                  'maxT' : 4},
                     'Fractional 0.10' :  {'filename' : 'euler_brownian_010/euler_brownian_1024/brownianeuler_cube_structure_cube_{p}.000000_{t}.nc',
                                  'nSaves' : 16,
                                  'maxT' : 4},
                     'Fractional 0.75' :  {'filename' : 'euler_brownian_075_time/euler_brownian_1024/brownianeuler_cube_structure_cube_{p}.000000_{t}.nc',
                                  'nSaves' : 16,
                                  'maxT' : 4},
                     'Shockvortex' : {'filename' : 'shockvortex/shockvortex_1024/shockvortex_cube_structure_cube_{p}.000000_{t}.nc',
                                  'nSaves' : 16,
                                  'maxT' : 4},
                     
                    }

variables = ['rho','mx','my','E']

P = [1,2,3]
for p in P:
    for name in evolutionDatasets.keys():
    
        exponents = []
        times = []
        nSaves = evolutionDatasets[name]['nSaves']
        for t in range(0, nSaves+1, nSaves//8):
            dVar = zeros(32)
            H = linspace(0,32./1024.,32)
            for variable in variables:
            
                filename = evolutionDatasets[name]['filename'].format(p=p,t=t)
                
                d = getlineintegral(filename, variable, 1024)
                
                
                dVar += d
            dVar = dVar**(1.0/p)
            
            poly = polyfit(log(H[1:]), log(dVar[1:]),1)
            exponents.append(poly[0])
            times.append(t*evolutionDatasets[name]['maxT']/float(nSaves))
            
            
            
        plt.plot(times,exponents, '--o',label='%s, $p=%d$' % (name, p))
    plt.legend()
    plt.grid('on')
    plt.xlabel('Time')
    plt.ylabel('Exponent')
    plt.ylim([-0.3,1])
    plt.yticks(linspace(0,1,11))
    showAndSave('exponent_evolution_%d' % p)
    